# Bootstrap generate album pages with tags (vertically alligned)

Data Sources
- https://www.flaticon.com

### Imports

In [1]:
import os
import pandas
import bs4
import math
import datetime
import random
from numpy.random import randint as ri

## Setup

In [2]:
lorem = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
lorem_parts = lorem.split()

def return_random_text(lower, upper):
    # Random number of words between 0-30
    num_words = ri(lower, upper)
    words = [lorem_parts[ri(0, len(lorem_parts))] for num in range(num_words)]
    paragraph = ' '.join(words)
    return paragraph.capitalize()

### Google fonts dictionary

In [3]:
google_fonts = {"bootstrap":{"link":'',
                            "specify":'',
                            "url":None},
                
                "raleway":{"link":'<link href="https://fonts.googleapis.com/css?family=Raleway" rel="stylesheet">',
                          "specify":"font-family: 'Raleway', sans-serif;",
                          "url":"https://fonts.google.com/specimen/Raleway?selection.family=Raleway"},
                
                "open_sans":{"link":'<link href="https://fonts.googleapis.com/css?family=Open+Sans" rel="stylesheet">',
                        "specify":"font-family: 'Open Sans', sans-serif;",
                       "url":"https://fonts.google.com/specimen/Open+Sans?selection.family=Open+Sans"},
                
                "roboto":{"link":'<link href="https://fonts.googleapis.com/css?family=Roboto" rel="stylesheet">',
                         "specify":"font-family: 'Roboto', sans-serif;",
                         "url":"https://fonts.google.com/specimen/Roboto?selection.family=Roboto"},
                
    "ubuntu":{"link":'<link href="https://fonts.googleapis.com/css?family=Ubuntu" rel="stylesheet"> ',
                         "specify":"font-family: 'Ubuntu', sans-serif;",
                         "url": "https://fonts.google.com/specimen/Ubuntu?selection.family=Ubuntu"}}

### Set font for website

In [4]:
selected_font = 'ubuntu'

if selected_font not in google_fonts.keys():
    print('The font chosen is not available. Please choose one from: \n{}'.format(list(google_fonts.keys())))
else:
    print('Font set to {}'.format(selected_font))
    
    set_link = google_fonts[selected_font]['link']
    set_style = google_fonts[selected_font]['specify']

Font set to ubuntu


### Set html colour codes 

In [5]:
jumbotron_background = '#1d91c0'
jumbotron_foreground = '#FFFFFF'
outline_button = 'dark'
card_header_foreground = '#44494a'
card_header_background = '#FFFFFF'
navbar_background = '#414242'
footer_background = '#414242'
badge_pill_background = '#1d91c0'
badge_pill_foreground  = '#FFFFFF'

### Set SVG flag, items per page

In [6]:
items_per_page = 12

svg = True

if svg == True:
    visual_tag_start = '<object class = "p-1" height=100% width=100% type="image/svg+xml" data="'
    visual_tag_end = '">Your browser does not support SVG</object>'
else:
    visual_tag_start = '<img class="card-img-top p-3" src="'
    visual_tag_end = '" alt="Card image cap">'

### Set enabled/disabled variables used to switch buttons between enabled and disabled modes

In [7]:
nothing = ''
disabled = 'disabled'

### Set Data Path and html path

In [8]:
data_path = './data/pokemon_svg/'
html_path = './'

## Read in files

### Read filenames from data_path variable

In [9]:
all_filenames = [file for file in os.listdir(data_path) if 'DS_Store' not in file]

### Store filenames in dataframe

In [10]:
data = pandas.DataFrame()
data["filename"] = pandas.Series(all_filenames)

### Create tags for testing purposes
- Create list of possible tags, this is arbitrary
- For each row in dataset containing filenames, calculate a random number (number of tags for this particular file), then for each increment of this random number assign a random tag from test_tags list

In [11]:
test_tags = ['blue', 'red', 'yellow', 'purple', 'grey', 'white', 'black', 'teal', 'orange', 'silver', 'pink', 'brown']

def generate_tags():
    tags = ''
    num_tags = ri(2,8) 
    tag_list = [test_tags[ri(0, len(test_tags)-1)] for tag in range(num_tags)]
    tags = ','.join(tag_list)
    return tags

In [12]:
generate_tags()

'purple,silver,white,blue,grey,pink'

In [13]:
for index in range(data.shape[0]):
    data.loc[index, "tag"] = generate_tags()

### Extract unique tags from dataframe

In [14]:
tag_string = ','.join(data['tag'].tolist())

In [15]:
tags_including_dupes = [tag for tag in tag_string.split(',') if len(tag)> 0]

In [16]:
no_dupes = sorted(list(set(tags_including_dupes)))

### Create dictionary to store indices for each tag
- Create the dictionary to store each unique tag, tags will be used to filter items
- Iterate through and its associated indices (each index for each filename associated with that tag)
- Finally add an 'index' key with all indices, this will be used as an unfiltered set of pages
- Preview dictionary to validate tag/index assignment

In [17]:
from collections import defaultdict

tag_storage = defaultdict(list)

for tag in no_dupes:
    for index in range(data.shape[0]):
        current_tag = data.loc[index, "tag"]
        if tag in current_tag:
            tag_storage[tag].append(index)

tag_storage["index"] = list(data.index)

print(tag_storage)

defaultdict(<class 'list'>, {'black': [0, 4, 5, 6, 9, 11, 13, 14, 15, 17, 19, 23, 27, 31, 35, 36, 41, 46, 47, 48, 52, 55, 57, 59, 63, 64, 66, 69, 70, 81, 82, 87, 89, 95, 97], 'blue': [1, 3, 4, 8, 10, 16, 21, 22, 26, 28, 29, 32, 34, 45, 48, 49, 53, 54, 56, 59, 60, 63, 64, 65, 69, 71, 74, 77, 78, 79, 80, 83, 84, 90, 93, 99], 'grey': [2, 4, 5, 6, 7, 16, 17, 18, 22, 24, 26, 27, 29, 36, 42, 44, 50, 53, 55, 56, 58, 63, 64, 72, 76, 78, 82, 84, 87, 90, 92, 95, 98, 99], 'orange': [1, 4, 5, 6, 7, 13, 14, 15, 21, 26, 27, 36, 38, 40, 47, 48, 49, 50, 52, 64, 69, 72, 73, 75, 77, 79, 81, 82, 83, 99], 'pink': [2, 7, 13, 18, 21, 33, 37, 39, 45, 47, 51, 54, 56, 57, 58, 61, 68, 70, 72, 73, 74, 79, 87, 90, 94], 'purple': [3, 7, 9, 11, 20, 27, 28, 34, 35, 36, 37, 41, 46, 49, 54, 56, 59, 64, 67, 69, 70, 73, 75, 76, 80, 81, 85, 86, 88, 93], 'red': [0, 1, 6, 7, 12, 15, 23, 28, 34, 36, 43, 47, 48, 50, 54, 56, 57, 59, 60, 61, 62, 64, 67, 70, 72, 74, 75, 77, 78, 79, 85, 91, 97, 98], 'silver': [6, 8, 14, 16, 20, 

## Templates

### Jumbotron

In [18]:
jumbotron_home = '''
<div class="jumbotron jumbotron-fluid text-center m-0" style = "color: '''+jumbotron_foreground+'''; background-color:'''+jumbotron_background+''';">
      <div class="container">
        <h1 class="display-2">
          Daniels Generated Album
        </h1>
        <p class="lead">
          Something short and leading about the collection below—its contents, the creator, etc. Make it short and sweet, but not too short so folks don't simply skip over it entirely.
        </p>
        <p>
          <a class="btn btn-outline-light my-2" href="#">
          Primary
          </a>
          <a class="btn btn-outline-light my-2" href="#">
          Secondary
          </a>
        </p>
      </div>
    </div>
'''

jumbotron_empty = ''

### Pagination - Variable

In [19]:
pagination = '<div class = "text-center pb-4 text-muted bg-light">\
  <div class = "btn-group">\
    <a href = "{}{}.html" class = "btn btn-outline-'+outline_button+' {}">First</a>\
    <a href = "{}{}.html" class = "btn btn-outline-'+outline_button+' {}">Previous</a>\
    <a class = "btn btn-outline-'+outline_button+' active">Page {}</a>\
    <a href = "{}{}.html" class = "btn btn-outline-'+outline_button+' {}">Next</a>\
    <a href = "{}{}.html"class = "btn btn-outline-'+outline_button+' {}">Last</a>\
  </div>\
</div>'

pagination2 = '''
<div class = "text-center pb-4 text-muted bg-light">
  <div class = "btn-group">
    <a class = "btn btn-outline-'+outline_button+' active">Page {}</a>
  </div>
</div>
'''

### Skeleton

In [20]:
skeleton = '''
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8"/>
    <meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
    <meta content="" name="description"/>
    <meta content="" name="author"/>
    <link href="../../../../favicon.ico" rel="icon"/>
    <title>
      Daniels Generated Gallery
    </title>
    <!-- Bootstrap CSS -->
    ''' + set_link + '''
    <link crossorigin="anonymous" href="https://stackpath.bootstrapcdn.com/bootstrap/4.1.1/css/bootstrap.min.css" integrity="sha384-WskhaSGFgHYWDcbwN70/dfYBj47jz9qbsMId/iRN3ewGhXQFZCSftd1LZCfmhktB" rel="stylesheet"/>
    <!-- Custom styles for this template -->
    <link href="album.css" rel="stylesheet"/>
  </head>
  <body class = "bg-light" style = "'''+set_style+'''">
    <header style = "background-color:'''+navbar_background+''';">
       <div class="container">
        <nav class="navbar navbar-expand-lg navbar-dark py-1" style = "background-color:'''+navbar_background+''';">
         <a class="navbar-brand" href="#">
          <img alt="" class="d-inline-block" height="36" src="logo/analytics.svg" width="36"/>
         </a>
         <button aria-controls="navbarColor03" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler collapsed" data-target="#navbarColor03" data-toggle="collapse" type="button">
          <span class="navbar-toggler-icon">
          </span>
         </button>
         <div class="navbar-collapse collapse" id="navbarColor03" style="">
          <ul class="navbar-nav mr-auto">
           <li class="nav-item active">
            <a class="nav-link" href="index0.html">
             Home
             <span class="sr-only">
              (current)
             </span>
            </a>
           </li>
           <li class="nav-item">
            <a class="nav-link" href="#">
             About
            </a>
           </li>
           <li class="nav-item">
            <a class="nav-link" href="#">
             Donate
            </a>
           </li>
           <li class="nav-item">
            <a class="nav-link" href="#">
             Community
            </a>
           </li>
           <li class="nav-item">
            <a class="nav-link" href="#">
             Contact Us
            </a>
           </li>
          </ul>
         </div>
    </nav>
   </div>
  </header>
    
    
    {}
    
    
    
    <div class="album bg-light">
      <div class="container">
        <div class="row mt-3">
          
          {}
        
        
        </div>
      </div>
    </div>
    
    {}
    
    <footer class="text-light py-3" style = "background-color: '''+footer_background+''';">
      <div class="container">
        <span>&copy; Copyright DanielsGenerator</span>
        <span class="float-right">
        <a href="#">
        Back to top
        </a>
        </span>
      </div>
    </footer>
    <!-- Optional JavaScript -->
    <!-- jQuery first, then Popper.js, then Bootstrap JS -->
    <script crossorigin="anonymous" integrity="sha384-q8i/X+965DzO0rT7abK41JStQIAqVgRVzpbzo5smXKp4YfRvH+8abtTE1Pi6jizo" src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
    <script crossorigin="anonymous" integrity="sha384-ZMP7rVo3mIykV+2+9J3UJ46jBk0WLaUAdn689aCwoqbBJiSnjAK/l8WvCWPIPm49" src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.3/umd/popper.min.js"></script>
    <script crossorigin="anonymous" integrity="sha384-smHYKdLADwkXOn1EmN1qk/HfnUcbVRZyYmZ4qpPea6sjB/pTJ0euyQp0Mk8ck+5T" src="https://stackpath.bootstrapcdn.com/bootstrap/4.1.1/js/bootstrap.min.js"></script>
  </body>
</html>
'''

## Delete all .html files in directory before Process

In [21]:
[os.remove(html_path + file) for file in os.listdir(html_path) if '.html' in file]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Process


- **ITERATE THROUGH EACH TAG `tag_storage` (the higest level)**
    - Store `tag` variable, `page_prefix` variable and `subset_indices`
    - Work out total items within the current `tag`
    - Work out `excess_items`, to determine how many pages will be required for current `tag`
    - Workout how many items in the `dic` tag list of indices will be included in each page using `items_per_page` and `total_pages variable`
    - Calculate `first_index` and `last_index`


- **ITERATE THROUGH EACH PAGE IN `nested_page_indices`** (second level, represents html pages, containing `items_per_page` items)
    - Calculate `previous_index` and `next_index`, used for pagination linking
    - Calculate pagination html
    - Calculate which jumbotron variable to use
    - Create column variables



- **ITERATE THROUGH EACH PAGE IN `page_items`** (lowest level, represents filenames)
    - Calculate `filename` and `filetags` variables
    - Calculate html block using `filetags` variable
    - Calculate the body html, this is comprised of column variables, each storing a dynamic amount of card html code

In [22]:
#ITERATE THROUGH EACH TAG IN DICTIONARY

for tag in tag_storage.keys():
    
    print('Processing tag_storage key:', tag)
    
    page_prefix = tag
    subset_indices = tag_storage[tag]

    #WORK OUT TOTAL ITEMS
    total_items = len(tag_storage[tag])
    
    #WORK OUT HOW MUCH EXCESS ITEMS
    excess_items = total_items % items_per_page
    
    # CALCULATE HOW MANY PAGES WILL BE REQUIRED
    if excess_items > 0:
        total_pages = int(math.floor(total_items/items_per_page) + 1)
    else:
        total_pages = int(total_items/items_per_page)
        
    index_starter = 0
    nested_page_indices = []
    
    # CALCULATE GROUPINGS OF FILES PER PAGE AND STORE IN A LIST OF LISTS
    if total_pages > 0:
        for index in range(total_pages):
            small_list = subset_indices[index_starter: index_starter + items_per_page]
            nested_page_indices.append(small_list)
            index_starter = index_starter + items_per_page
    
    #CALCULATE INDEX FOR FIRST, LAST PAGES
    first_index = 0
    last_index = len(nested_page_indices) - 1
    
    #ITERATE THROUGH EACH PAGE IN NESTED_PAGE_INDICES
    
    for page_index, page_items in enumerate(nested_page_indices):
    
        # WORK OUT INDEX FOR PAGES LATERALLY
        previous_index = page_index -1
        next_index = page_index +1
        page_number = page_index + 1

        #PAGINATION
        if total_pages == 1:
            pagination_final = pagination2.format(page_number)
        else:
            if index ==0:
                pagination_final = pagination.format(page_prefix, first_index,disabled,page_prefix,  previous_index, disabled, page_number,
                                                    page_prefix, next_index, nothing, page_prefix, last_index, nothing)
            elif index == last_index:
                pagination_final = pagination.format(page_prefix, first_index, nothing, page_prefix, previous_index, nothing, page_number,
                                                    page_prefix, next_index, disabled, page_prefix, last_index, disabled)
            else:
                pagination_final = pagination.format(page_prefix,first_index, nothing,page_prefix, previous_index, nothing, page_number,
                                                    page_prefix, next_index, nothing, page_prefix, last_index, nothing)

        if page_index == 0:
            jumbotron_final = jumbotron_home
        else:
            jumbotron_final = jumbotron_empty

        item_number = 1
        
        # CREATE EMPTY CARDS
            
        cards = ''
        
        # FILE LEVEL LOOP
        for index2, file_index in enumerate(page_items):
            
            filename = data.loc[file_index, 'filename']
            filetags = data.loc[file_index, 'tag']

            tags_split = filetags.split(',')
            tags_html_block = ''

            for tag_name in tags_split:
                tags_html_block = tags_html_block + \
                '<a href = "'+tag_name+'0.html" class = "badge badge-pill" style = "color:'+badge_pill_foreground+' ;background-color:'+badge_pill_background+';">'+tag_name+'</a>'

            item_number += 1
            
            # Generate random text
            random_text = return_random_text(30,40)
            a_subtitle = return_random_text(6,8)
            cards = cards + '\
            <section class = "col-lg-4 col-md-6 mb-4">\
                <div class="card">\
                    <div class = "card-header p-3" style = "background-color: '+card_header_background+';">\
                        <h5 class = "p-0 m-0">Source: '+filename+'</h5>\
                        <div class = "card-subtitle p-0 m-0 text-muted">'+a_subtitle+'</div>\
                    </div>\
                      ' + visual_tag_start + data_path + filename + visual_tag_end + '\
                    <div class="card-body">\
                      <p class="card-text">' + random_text + '</p>\
                        <div class="btn-group p-0 m-0">\
                          <a class="btn btn-sm btn-outline-'+outline_button+'" href="'+data_path + filename+'" role="button">View</a>\
                          <a class="btn btn-sm btn-outline-'+outline_button+'" href = "'+data_path + filename+'" download = "'+filename+'" role="button">Download</a>\
                        </div>\
                    </div>\
                    <div class = "p-3 bg-light">\
                      '+tags_html_block+'\
                    </div>\
                  </div>\
                </section>'
            
        # BUILD FULL HTML CODE ON PAGE LEVEL
        full = skeleton.format(jumbotron_final, 
                               cards, 
                               pagination_final)

        # PRETTIFY/FORMAT THE FULL HTML CODE
        full_soup = bs4.BeautifulSoup(full, 'lxml')
        full_pretty = full_soup.prettify()

        #EXPORT HTML
        full_export_path = './' + tag + str(page_index) + '.html'
        with open(full_export_path, "w") as fileobject:
            fileobject.write(full_pretty)
            print('\tFile successfully written to', full_export_path)

        fileobject.close()

Processing tag_storage key: black
	File successfully written to ./black0.html
	File successfully written to ./black1.html
	File successfully written to ./black2.html
Processing tag_storage key: blue
	File successfully written to ./blue0.html
	File successfully written to ./blue1.html
	File successfully written to ./blue2.html
Processing tag_storage key: grey
	File successfully written to ./grey0.html
	File successfully written to ./grey1.html
	File successfully written to ./grey2.html
Processing tag_storage key: orange
	File successfully written to ./orange0.html
	File successfully written to ./orange1.html
	File successfully written to ./orange2.html
Processing tag_storage key: pink
	File successfully written to ./pink0.html
	File successfully written to ./pink1.html
	File successfully written to ./pink2.html
Processing tag_storage key: purple
	File successfully written to ./purple0.html
	File successfully written to ./purple1.html
	File successfully written to ./purple2.html
Processi